In [1]:
from timetable import timetable
from passenger import simu
import sys
sys.argv = ['debug.ipynb', 
            '--card_data_path', '../card_info.csv', 
            '--bus_data_path', '../公交数据.xlsx',
            '--road_line_station_setting_path', '../datas/road_line_station_structure_setting.json',
            '--station_distance_rank_path', '../datas/station_pair_distance_rank_index.csv',
            '--id_2_station_path', '../datas/id_2_station.json',
            '--station_2_id_path', '../datas/station_2_id.json']
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--card_data_path',default='./card_info.csv')
parser.add_argument('--bus_data_path',default= './公交数据.xlsx')
parser.add_argument('--road_line_station_setting_path',default= './datas/road_line_station_structure_setting.json')
parser.add_argument('--station_distance_rank_path',default= './datas/station_pair_distance_rank_index.csv')
parser.add_argument('--id_2_station_path',default= './datas/id_2_station.json')
parser.add_argument('--station_2_id_path',default= './datas/station_2_id.json')
myargs = parser.parse_args()
mytime_table = timetable(\
    card_data_path = myargs.card_data_path,\
    bus_data_path = myargs.bus_data_path,\
    road_line_station_setting_path = myargs.road_line_station_setting_path,\
    station_distance_rank_path = myargs.station_distance_rank_path,\
    id_2_station_path = myargs.id_2_station_path,\
    station_2_id_path = myargs.station_2_id_path)
this_simu = simu(\
    mytime_table,\
    card_data_path = myargs.card_data_path,\
    bus_data_path = myargs.bus_data_path,\
    road_line_station_setting_path = myargs.road_line_station_setting_path,\
    station_distance_rank_path = myargs.station_distance_rank_path,\
    id_2_station_path = myargs.id_2_station_path,\
    station_2_id_path = myargs.station_2_id_path,\
    time_accurate = 8,\
    start_hour = 8,\
    start_minute = 23)

/root/mnt/bus_corr/all_code/timetable.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip_1_0929['trip'] = trip_1_0929.\
/root/mnt/bus_corr/all_code/timetable.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp1.rename(columns = transfer,inplace=True)
/root/mnt/bus_corr/all_code/timetable.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2.rename(columns = t

In [ ]:
this_simu.

In [ ]:
this_simu.now_minute += 1
if this_simu.now_minute == 60:
    this_simu.now_minute = 0
    this_simu.now_hour += 1
if this_simu.now_hour == 24:
    this_simu.now_hour = 0
station_id_list = list(this_simu.id_2_station.keys())
now_conditions_df=pd.DataFrame({'this_od_trip_start_station':station_id_list,\
                                        'hour':[this_simu.now_hour]*len(station_id_list),\
                                        'minute_group':[this_simu.now_minute//this_simu.time_accurate]*len(station_id_list)})
now_user_all_possible_route_df = now_conditions_df.merge(\
            this_simu.different_time_user_produce, \
            left_on=['this_od_trip_start_station', 'hour', 'minute_group'],\
            right_on = ['global_station_id','hour','minute_group'], how='left').fillna(0)
        #now_user_all_possible_route_df['zip_target_station_line'] = now_user_all_possible_route_df.apply(lambda x:zip_target_station_line_func(x),axis=1)
now_station_routes_and_nums = \
                    now_user_all_possible_route_df.merge(this_simu.different_time_user_produce_nums,\
                                     left_on=['this_od_trip_start_station', 'hour', 'minute_group'],\
                                     right_on = ['this_od_trip_start_station', 'hour', 'minute_group'],\
                                     how='left').fillna(0)
now_station_routes_and_nums['users'] = now_station_routes_and_nums.apply(lambda x:uniform_sample(x),axis=1)
now_passengers = []
for row_index in range(len(now_station_routes_and_nums)):
    if len(now_station_routes_and_nums[row_index:row_index+1]['users'].values[0])!=0:
        #print(merged_df[row_index:row_index+1]['users'].values[0])
        start_station = now_station_routes_and_nums[row_index:row_index+1]['this_od_trip_start_station'].values[0]
        start_time = str(this_simu.now_hour)+':'+str(this_simu.now_minute)+':00'
        start_time = pd.to_datetime(start_time, format='%H:%M:%S', errors='coerce').dt.time
        #start_time = now_station_routes_and_nums[row_index:row_index+1]['minute_group'].values[0]
        for user_instance in now_station_routes_and_nums[row_index:row_index+1]['users'].values[0]:
            now_passengers.append({\
                        'start_station':user_instance['this_od_trip_start_station'],\
                        'start_time':start_time,\
                        'target_station':user_instance['this_od_end_station'],\
                        'target_line':user_instance['this_od_by_line']})
        #根据时间表为每个乘客添加时间表
for user in now_passengers:
    if user['start_station'] < user['target_station'] and this_simu.time_schedule.time_table[user['target_line']]:
        time_df = this_simu.time_schedule.time_table[user['target_line']]
        time_df_column = time_df.loc[:,user['start_station']]
        row_index = time_df_column[time_df_column>=user['start_time']].index.max()
        column_index = time_df.columns.get_loc(user['start_station'])
        user['time_table'] = time_df.iloc[row_index,column_index:]
        this_simu.total_passengers.append(user)
    else:
        continue

In [ ]:
this_simu.step()
this_simu.observe()